In [1]:
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
# Loading data
file_path = Path("../Resources/needs_input.csv")
cleaned_df = pd.read_csv(file_path)
cleaned_df.head()

In [ ]:
# Define features set
X = cleaned_df.copy()
X.drop("Heart_Disease", axis=1, inplace=True)
X.head()

In [ ]:
# Define target vector
y = cleaned_df["Heart_Disease"].values.reshape(-1, 1)
y[:5]

In [ ]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Creating StandardScaler instance
scaler = StandardScaler()

In [ ]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

In [ ]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Fitting the Decision Tree Model

In [ ]:
# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

In [ ]:
# Fitting the model
model = model.fit(X_train_scaled, y_train)

In [ ]:
## Making Predictions Using the Tree Model

In [ ]:
# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

In [ ]:
## Model Evaluation

In [ ]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))